In [21]:
import numpy as np
import os
from PIL import Image

# CIFAR-10 class names to integer labels mapping
label_map = {
    'airplane': 0,
    'automobile': 1,
    'bird': 2,
    'cat': 3,
    'deer': 4,
    'dog': 5,
    'frog': 6,
    'horse': 7,
    'ship': 8,
    'truck': 9
}

def load_images_from_folder(folder):
    images = []
    labels = []
    for label in sorted(os.listdir(folder)):
        label_folder = os.path.join(folder, label)
        if not os.path.isdir(label_folder):
            continue  # Process directories
        for image_file in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_file)
            with Image.open(image_path) as img:
                img = img.resize((32, 32))  # Resize to 32x32
                img_array = np.asarray(img, dtype=np.float32) / 255.0  # Normalize pixel values
                images.append(img_array.flatten())  # Flatten the image
                labels.append(label_map[label])  # Convert class names to integers
    images = np.array(images).T  # Shape: (3072, num_samples), CIFAR-10's 32x32x3 structure
    labels = np.array(labels).reshape(1, -1)  # Shape: (1, num_samples), now as integers
    return images, labels

# Load dataset
X_train, y_train = load_images_from_folder('data/train')



class Activation:
    @staticmethod
    def relu(z):
        return np.maximum(0, z)

    @staticmethod
    def d_relu(z):
        return np.where(z > 0, 1.0, 0.0)

    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def d_sigmoid(z):
        s = Activation.sigmoid(z)
        return s * (1 - s)

class Parameters:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size) * 0.1
        self.biases = np.zeros((output_size, 1))

class Layer:
    def __init__(self, input_size, output_size, activation):
        self.params = Parameters(input_size, output_size)
        self.activation = activation
        self.z = None
        self.a = None
        self.a_prev = None

    def forward(self, a_prev):
        self.a_prev = a_prev
        self.z = np.dot(self.params.weights, a_prev) + self.params.biases
        self.a = getattr(Activation, self.activation)(self.z)
        return self.a

    def backward(self, dA, learning_rate):
        m = self.a_prev.shape[1]
        if self.activation == 'relu':
            dZ = dA * Activation.d_relu(self.z)
        elif self.activation == 'sigmoid':
            dZ = dA * Activation.d_sigmoid(self.z)
        dW = np.dot(dZ, self.a_prev.T) / m
        dB = np.sum(dZ, axis=1, keepdims=True) / m
        dA_prev = np.dot(self.params.weights.T, dZ)

        self.params.weights -= learning_rate * dW
        self.params.biases -= learning_rate * dB
        return dA_prev

class DeepNeuralNetwork:
    def __init__(self, input_size=3072):
        self.layers = [
            Layer(input_size, 10, 'relu'),
            Layer(10, 8, 'relu'),
            Layer(8, 8, 'relu'),
            Layer(8, 4, 'relu'),
            Layer(4, 1, 'sigmoid')
        ]

    def forward_propagation(self, X):
        a = X
        for layer in self.layers:
            a = layer.forward(a)
        return a

    def compute_loss(self, y, y_hat):
        m = y.shape[1]
        loss = -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
        return loss

    def backward_propagation(self, y, y_hat, learning_rate):
        epsilon = 1e-7
        dA = -(y / np.clip(y_hat, epsilon, 1 - epsilon)) + ((1 - y) / np.clip(1 - y_hat, epsilon, 1 - epsilon))
        for layer in reversed(self.layers):
            dA = layer.backward(dA, learning_rate)


    def train(self, X, y, learning_rate, n_iterations):
        epsilon = 1e-7
        for i in range(n_iterations):
            y_hat = self.forward_propagation(X)
            y_hat_clipped = np.clip(y_hat, epsilon, 1 - epsilon)
            loss = -np.mean(y * np.log(y_hat_clipped) + (1 - y) * np.log(1 - y_hat_clipped))
            if i % 100 == 0:
                print(f"Iteration: {i}, Loss: {loss:.4f}")

            # Call backward_propagation with y, y_hat_clipped, and learning_rate
            self.backward_propagation(y, y_hat_clipped, learning_rate)


    
    def print_model(self):
        for i, layer in enumerate(self.layers, 1):
            print(f"Layer {i}: {layer.params.weights.shape[0]} neurons, Activation: {layer.activation}")


# Initialize and train the network
dnn = DeepNeuralNetwork(input_size=3072)
dnn.train(X_train, y_train, learning_rate=0.001, n_iterations=1000)
dnn.print_model()




Iteration: 0, Loss: 0.6968
Iteration: 100, Loss: -0.8819
Iteration: 200, Loss: -2.3964
Iteration: 300, Loss: -3.8886
Iteration: 400, Loss: -5.4925
Iteration: 500, Loss: -11.9798
Iteration: 600, Loss: -56.3560
Iteration: 700, Loss: -56.3910
Iteration: 800, Loss: -56.3998
Iteration: 900, Loss: -56.4051
Layer 1: 10 neurons, Activation: relu
Layer 2: 8 neurons, Activation: relu
Layer 3: 8 neurons, Activation: relu
Layer 4: 4 neurons, Activation: relu
Layer 5: 1 neurons, Activation: sigmoid
